# C++ Cling Examples for STM32F767X board

![Image](img/board.jpg)

Load [REMCU](https://remotemcu.github.io) shared libray

In [1]:
.L libremcu

Add path with header files

In [2]:
.I remcu_include/

Including necessary header files. The [“remcu.h”](remcu_include/remcu.h) header must be always included before any MCU header files.

In [3]:
#include "remcu.h"

Connection to a debug server. The [**REMCU**](https://remotemcu.github.io/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/).
There is using the [remcu_connect2OpenOCD](https://remotemcu.github.io/api-v1-0#remcu_getVersion) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)

Before that, plug stm32 board with debugger to PC

run openocd utility in command line:
```
openocd -f board/stm32f7discovery.cfg
```
Result:
![Image](img/openocd_unix.png)

In [4]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const int timeout_sec = 3; // It can not be negative

In [5]:
remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

*******************************************************************
It is a Community license.

REMCU lib can freely be used for non-profit educational purposes without any 
limitation. Simply download and install it and start development of your new 
projects. No registration or activation process is required.

If you are an individual working on your own devices to sell or for any other 
purpose, you may use the software to develop and test it without any limitation.

Commercial and Extended licenses can be gotten through the license request dialog.
email: en.chormonov@gmail.com

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LI

true

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server  etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [6]:
//const uint16_t default_openocd_gdb_port = 3333;
//const uint16_t default_jlink_port = 2331;
//const uint16_t default_stlink_port = 61234;
//const uint16_t custom_port = ...;
//remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port /*default_jlink_port*/ /*default_stlink_port*/, timeout_sec)

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page) 
    
If connection were success... Jupyter Output:
![Image](img/success_connection_for_unix.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)):

In [7]:
remcu_resetRemoteUnit(__HALT)

true

HAL Low Level header files:

In [8]:
#include "stm32f7xx_ll_adc.h"
#include "stm32f7xx_ll_dma.h"
#include "stm32f7xx_ll_tim.h"
#include "stm32f7xx_ll_dac.h"
#include "stm32f7xx_ll_gpio.h"
#include "stm32f7xx_ll_bus.h"
#include "stm32f7xx_ll_rcc.h"
#include <iostream>

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (system_stm32f7xx.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/system_stm32f7xx.c file

In [9]:
SystemInit()

Simple Clock configuration:

In [10]:
 /* Enable HSI and wait for HSI ready*/
  LL_RCC_HSI_Enable();

  while(LL_RCC_HSI_IsReady() != 1)

In [11]:
/* Select HSI as system clock */
/* Wait for HSI switched */
  LL_RCC_SetSysClkSource(LL_RCC_SYS_CLKSOURCE_HSI);

### ADC example

![adc_head](img/adc_head.png )

>The code below have been taken from **ADC3_DMA** ([main.c](../STM32F4-Discovery_FW_V1.1.0/Project/Peripheral_Examples/ADC3_DMA/main.c)) example of [Standard Peripheral Library(SPL) for STM32F4DISCOVERY](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-library-expansion/stsw-stm32068.html#overview).
DMA code was dropped.

In [12]:
/*## Configuration of GPIO used by ADC channels ############################*/
  
  /* Note: On this STM32 device, ADC1 channel 4 is mapped on GPIO pin PA.04 */ 
  
  /* Enable GPIO Clock */
  LL_AHB1_GRP1_EnableClock(LL_AHB1_GRP1_PERIPH_GPIOA);
  
  /* Configure GPIO in analog mode to be used as ADC input */
  LL_GPIO_SetPinMode(GPIOA, LL_GPIO_PIN_4, LL_GPIO_MODE_ANALOG);
  
  /*## Configuration of ADC ##################################################*/
  
  /*## Configuration of ADC hierarchical scope: common to several ADC ########*/
  
  /* Enable ADC clock (core clock) */
  LL_APB2_GRP1_EnableClock(LL_APB2_GRP1_PERIPH_ADC1);
  
  /* Note: Hardware constraint (refer to description of the functions         */
  /*       below):                                                            */
  /*       On this STM32 serie, setting of these features are not             */
  /*       conditioned to ADC state.                                          */
  /*       However, in order to be compliant with other STM32 series          */
  /*       and to show the best practice usages, ADC state is checked.        */
  /*       Software can be optimized by removing some of these checks, if     */
  /*       they are not relevant considering previous settings and actions    */
  /*       in user application.                                               */
  if(__LL_ADC_IS_ENABLED_ALL_COMMON_INSTANCE() == 0)
  {
    /* Note: Call of the functions below are commented because they are       */
    /*       useless in this example:                                         */
    /*       setting corresponding to default configuration from reset state. */
    
    /* Set ADC clock (conversion clock) common to several ADC instances */
    LL_ADC_SetCommonClock(__LL_ADC_COMMON_INSTANCE(ADC1), LL_ADC_CLOCK_SYNC_PCLK_DIV2);
    
    /* Set ADC measurement path to internal channels */
    // LL_ADC_SetCommonPathInternalCh(__LL_ADC_COMMON_INSTANCE(ADC1), LL_ADC_PATH_INTERNAL_NONE);
    
    
  /*## Configuration of ADC hierarchical scope: multimode ####################*/
  
    /* Set ADC multimode configuration */
    // LL_ADC_SetMultimode(__LL_ADC_COMMON_INSTANCE(ADC1), LL_ADC_MULTI_INDEPENDENT);
    
    /* Set ADC multimode DMA transfer */
    // LL_ADC_SetMultiDMATransfer(__LL_ADC_COMMON_INSTANCE(ADC1), LL_ADC_MULTI_REG_DMA_EACH_ADC);
    
    /* Set ADC multimode: delay between 2 sampling phases */
    // LL_ADC_SetMultiTwoSamplingDelay(__LL_ADC_COMMON_INSTANCE(ADC1), LL_ADC_MULTI_TWOSMP_DELAY_1CYCLE);
    
  }
  
  
  /*## Configuration of ADC hierarchical scope: ADC instance #################*/
  
  /* Note: Hardware constraint (refer to description of the functions         */
  /*       below):                                                            */
  /*       On this STM32 serie, setting of these features are not             */
  /*       conditioned to ADC state.                                          */
  /*       However, ADC state is checked anyway with standard requirements    */
  /*       (refer to description of this function).                           */
  if (LL_ADC_IsEnabled(ADC1) == 0)
  {
    /* Note: Call of the functions below are commented because they are       */
    /*       useless in this example:                                         */
    /*       setting corresponding to default configuration from reset state. */
    
    /* Set ADC data resolution */
    // LL_ADC_SetResolution(ADC1, LL_ADC_RESOLUTION_12B);
    
    /* Set ADC conversion data alignment */
    // LL_ADC_SetResolution(ADC1, LL_ADC_DATA_ALIGN_RIGHT);
    
    /* Set Set ADC sequencers scan mode, for all ADC groups                   */
    /* (group regular, group injected).                                       */
    // LL_ADC_SetSequencersScanMode(ADC1, LL_ADC_SEQ_SCAN_DISABLE);
    
  }
  
  
  /*## Configuration of ADC hierarchical scope: ADC group regular ############*/
  
  /* Note: Hardware constraint (refer to description of the functions         */
  /*       below):                                                            */
  /*       On this STM32 serie, setting of these features are not             */
  /*       conditioned to ADC state.                                          */
  /*       However, ADC state is checked anyway with standard requirements    */
  /*       (refer to description of this function).                           */
  if (LL_ADC_IsEnabled(ADC1) == 0)
  {
    /* Set ADC group regular trigger source */
    LL_ADC_REG_SetTriggerSource(ADC1, LL_ADC_REG_TRIG_SOFTWARE);
    
    /* Set ADC group regular trigger polarity */
    // LL_ADC_REG_SetTriggerEdge(ADC1, LL_ADC_REG_TRIG_EXT_RISING);
    
    /* Set ADC group regular continuous mode */
    LL_ADC_REG_SetContinuousMode(ADC1, LL_ADC_REG_CONV_CONTINUOUS);
    
    /* Set ADC group regular conversion data transfer */
    // LL_ADC_REG_SetDMATransfer(ADC1, LL_ADC_REG_DMA_TRANSFER_NONE);
    
    /* Specify which ADC flag between EOC (end of unitary conversion)         */
    /* or EOS (end of sequence conversions) is used to indicate               */
    /* the end of conversion.                                                 */
    // LL_ADC_REG_SetFlagEndOfConversion(ADC1, LL_ADC_REG_FLAG_EOC_SEQUENCE_CONV);
    
    /* Set ADC group regular sequencer */
    /* Note: On this STM32 serie, ADC group regular sequencer is              */
    /*       fully configurable: sequencer length and each rank               */
    /*       affectation to a channel are configurable.                       */
    /*       Refer to description of function                                 */
    /*       "LL_ADC_REG_SetSequencerLength()".                               */
    
    /* Set ADC group regular sequencer length and scan direction */
    LL_ADC_REG_SetSequencerLength(ADC1, LL_ADC_REG_SEQ_SCAN_DISABLE);
    
    /* Set ADC group regular sequencer discontinuous mode */
    // LL_ADC_REG_SetSequencerDiscont(ADC1, LL_ADC_REG_SEQ_DISCONT_DISABLE);
    
    /* Set ADC group regular sequence: channel on the selected sequence rank. */
    LL_ADC_REG_SetSequencerRanks(ADC1, LL_ADC_REG_RANK_1, LL_ADC_CHANNEL_4);
  }
  
  
  /*## Configuration of ADC hierarchical scope: ADC group injected ###########*/
  
  /* Note: Hardware constraint (refer to description of the functions         */
  /*       below):                                                            */
  /*       On this STM32 serie, setting of these features are not             */
  /*       conditioned to ADC state.                                          */
  /*       However, ADC state is checked anyway with standard requirements    */
  /*       (refer to description of this function).                           */
  if (LL_ADC_IsEnabled(ADC1) == 0)
  {
    /* Note: Call of the functions below are commented because they are       */
    /*       useless in this example:                                         */
    /*       setting corresponding to default configuration from reset state. */
    
    /* Set ADC group injected trigger source */
    // LL_ADC_INJ_SetTriggerSource(ADC1, LL_ADC_INJ_TRIG_SOFTWARE);
    
    /* Set ADC group injected trigger polarity */
    // LL_ADC_INJ_SetTriggerEdge(ADC1, LL_ADC_INJ_TRIG_EXT_RISING);
    
    /* Set ADC group injected conversion trigger  */
    // LL_ADC_INJ_SetTrigAuto(ADC1, LL_ADC_INJ_TRIG_INDEPENDENT);
    
    /* Set ADC group injected sequencer */
    /* Note: On this STM32 serie, ADC group injected sequencer is             */
    /*       fully configurable: sequencer length and each rank               */
    /*       affectation to a channel are configurable.                       */
    /*       Refer to description of function                                 */
    /*       "LL_ADC_INJ_SetSequencerLength()".                               */
    
    /* Set ADC group injected sequencer length and scan direction */
    // LL_ADC_INJ_SetSequencerLength(ADC1, LL_ADC_INJ_SEQ_SCAN_DISABLE);
    
    /* Set ADC group injected sequencer discontinuous mode */
    // LL_ADC_INJ_SetSequencerDiscont(ADC1, LL_ADC_INJ_SEQ_DISCONT_DISABLE);
    
    /* Set ADC group injected sequence: channel on the selected sequence rank. */
    // LL_ADC_INJ_SetSequencerRanks(ADC1, LL_ADC_INJ_RANK_1, LL_ADC_CHANNEL_4);
  }
  
  
  /*## Configuration of ADC hierarchical scope: channels #####################*/
  
  /* Note: Hardware constraint (refer to description of the functions         */
  /*       below):                                                            */
  /*       On this STM32 serie, setting of these features are not             */
  /*       conditioned to ADC state.                                          */
  /*       However, in order to be compliant with other STM32 series          */
  /*       and to show the best practice usages, ADC state is checked.        */
  /*       Software can be optimized by removing some of these checks, if     */
  /*       they are not relevant considering previous settings and actions    */
  /*       in user application.                                               */
  if (LL_ADC_IsEnabled(ADC1) == 0)
  {
    /* Set ADC channels sampling time */
    LL_ADC_SetChannelSamplingTime(ADC1, LL_ADC_CHANNEL_4, LL_ADC_SAMPLINGTIME_480CYCLES);
  }

/*## Operation on ADC hierarchical scope: ADC instance #####################*/
  
  /* Note: Hardware constraint (refer to description of the functions         */
  /*       below):                                                            */
  /*       On this STM32 serie, setting of these features are not             */
  /*       conditioned to ADC state.                                          */
  /*       However, in order to be compliant with other STM32 series          */
  /*       and to show the best practice usages, ADC state is checked.        */
  /*       Software can be optimized by removing some of these checks, if     */
  /*       they are not relevant considering previous settings and actions    */
  /*       in user application.                                               */
  if (LL_ADC_IsEnabled(ADC1) == 0)
  {
    /* Enable ADC */
    LL_ADC_Enable(ADC1);
  }

  if (LL_ADC_IsEnabled(ADC1) == 0)
  {
      std::cerr << "Error: Invalid Initialization of ADC1" << std::endl;
  }

The function definition. This function does conversion and return the raw ADC value:

In [13]:
uint16_t ADC_value(){
    LL_ADC_REG_StartConversionSWStart(ADC1);
    return LL_ADC_REG_ReadConversionData12(ADC1);
}

ADC_value()

1034

Prepare canvas, which shows ADC data. [Template](https://github.com/QuantStack/xplot/blob/3399c4b96dffb1fec9d9c44de74d3d223bb04789/notebooks/xplot.ipynb)
> First install [xplot](https://github.com/QuantStack/xplot) or make sure you have it
<details>
  <summary>The kernel of notebook should be  <b>C++14</b> (clck here)</summary>
    
![Image](img/Cpp14.png)  

</details>

In [14]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

const size_t ADC_BUFF_SIZE = 300;
xpl::figure fig;
xpl::linear_scale sx, sy;
xpl::lines line(sx, sy);
std::vector<uint16_t> adc_data(ADC_BUFF_SIZE);
std::vector<uint16_t> view_data(0);
std::vector<int> x_line(ADC_BUFF_SIZE);
int i = 0;
for( auto & x : x_line){x = i++;}
line.x = x_line;
line.y = adc_data;
fig.add_mark(line);
    
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);
fig

A Jupyter widget

Plug a potentiometer into the PC4 pin and run the code below. Rotate the shaft and you will get the voltage versus time chart:

In [15]:
#include <unistd.h>

for(size_t i = 0; i < ADC_BUFF_SIZE; i++){
    adc_data[i] = ADC_value();
    view_data.resize(i+1);
    for(int j=0; j<= i; j++){
        view_data[j] = adc_data[j];
    }
    //plot ADC data
    line.y = view_data;
    usleep(100000);
}

In [ ]:
adc_data.clear();
adc_data.resize(ADC_BUFF_SIZE);
line.y = adc_data;